In [91]:
import pandas as pd
import numpy as np
import os

#Importando csv
df = pd.read_csv('../../data_sets/vendas_produtos_financeiros.csv', sep=';')
df.head()

,Produto,Quantidade,Preco_Unitario,Valor_Total,Data,Estado
0,Produto 10,5,6.03,30.15,05/10/2023 10:03,RS
1,Produto 2,9,20.48,184.32,17/03/2023 14:03,RJ
2,Produto 5,3,25.39,76.17,20/04/2023 09:03,BA
3,Produto 6,3,10.73,32.19,21/09/2023 15:03,AM
4,Produto 8,4,34.67,138.68,04/09/2023 01:03,SP


In [10]:
#Fazendo tratamento dos dados antes 
#verificando
df.isna()

,Produto,Quantidade,Preco_Unitario,Valor_Total,Data,Estado
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
2026,False,False,False,False,False,False
2027,False,False,False,False,False,False
2028,False,False,False,False,False,False
2029,False,False,False,False,False,False


In [92]:
#Verificando novamente
df.isna().sum()

Produto            5
Quantidade         0
Preco_Unitario    12
Valor_Total       10
Data               0
Estado             0
dtype: int64

In [9]:
#usando where
df.where(df['Preco_Unitario']>=1)

,Produto,Quantidade,Preco_Unitario,Valor_Total,Data,Estado
0,Produto 10,5.0,6.03,30.15,05/10/2023 10:03,RS
1,Produto 2,9.0,20.48,184.32,17/03/2023 14:03,RJ
2,Produto 5,3.0,25.39,76.17,20/04/2023 09:03,BA
3,Produto 6,3.0,10.73,32.19,21/09/2023 15:03,AM
4,Produto 8,4.0,34.67,138.68,04/09/2023 01:03,SP
...,...,...,...,...,...,...
2026,Produto 6,10.0,3.58,35.80,05/04/2023 19:03,SP
2027,Produto 4,8.0,20.38,163.04,07/03/2023 09:03,SP
2028,Produto 10,8.0,8.12,64.96,21/11/2022 21:03,PR
2029,Produto 3,2.0,17.09,34.18,01/04/2023 14:03,PR


In [16]:
#Criando outro dataframe só com NA ou null
df1 = df[df.isna().any(axis=1)]
df1.tail()

,Produto,Quantidade,Preco_Unitario,Valor_Total,Data,Estado
1855,Produto 3,6,NaN,109.20,16/03/2023 01:03,AM
1856,Produto 6,10,NaN,102.40,30/04/2023 11:03,RS
1857,Produto 7,3,NaN,102.09,31/03/2023 01:03,RS
1858,Produto 7,1,NaN,38.50,25/05/2023 16:03,BA
1859,Produto 7,8,NaN,332.48,29/10/2023 12:03,MG


In [93]:
#Verificando duplicados
duplicados = df.duplicated()
duplicados.sum()


np.int64(31)

In [112]:
#df = df.drop_duplicates()
nao_dup = df.duplicated()
nao_dup.sum()

np.int64(0)

In [97]:
#Duplicados tem que ser depois do tratamento dos dados NA
df['Valor_Total'] = df['Valor_Total'].where(
    df['Valor_Total'].notna(),  # Mantém os valores não NaN
    other=df['Quantidade'] * df['Preco_Unitario']  # Substitui NaN pelo produto
)



In [98]:
df['Preco_Unitario'] = df['Preco_Unitario'].where(
    df['Preco_Unitario'].notna(),  # Mantém os valores não NaN
    other=df['Valor_Total'] / df['Quantidade']  # Substitui NaN pelo produto
)

In [99]:
df.isna().sum()

Produto           5
Quantidade        0
Preco_Unitario    0
Valor_Total       0
Data              0
Estado            0
dtype: int64

In [100]:
df=df.dropna(subset=['Produto'])

In [111]:
df.isna().sum()

Produto           0
Quantidade        0
Preco_Unitario    0
Valor_Total       0
Data              0
Estado            0
dtype: int64

In [102]:
#Qual é a média do valor das vendas de todos os produtos destinados ao Estado de São Paulo (SP)
#Primeiro filtrar
df_sp = df[df["Estado"] == "SP"]
#Agora obter media
media_vendas_sp = df_sp["Valor_Total"].mean()
print(f"Média do valor das vendas para SP: R$ {media_vendas_sp:.2f}")

Média do valor das vendas para SP: R$ 111.33


In [110]:
#Qual é o desvio padrão da quantidade unitária para todos os produtos vendidos no Estado de Pernambuco (PE)?
df_pe = df[df['Estado'] == 'PE']

# Calcular o desvio padrão da coluna Quantidade
desvio_padrao_pe = df_pe['Quantidade'].std()

print(f"Desvio padrão da quantidade de produtos vendidos em PE: {desvio_padrao_pe:.2f}")

Desvio padrão da quantidade de produtos vendidos em PE: 2.88


In [104]:
#Valor maximo de venda no Estado de SC
df_sc = df[df["Estado"] == "SC"]
maximo_sc = df_sc["Valor_Total"].max()
print(f"Valor maximo de venda para SC: {maximo_sc:}")

Valor maximo de venda para SC: 356.2


In [105]:
#Calcule a soma da coluna "Quantidade" para todos os produtos e responda: qual produto teve o maior total de unidades vendidas?
total_por_produto = df.groupby('Produto')['Quantidade'].sum()
produto_mais_vendido = total_por_produto.idxmax()
maior_quantidade = total_por_produto.max()
print("Total de unidades por produto:")
print(total_por_produto)
print("\nProduto mais vendido:", produto_mais_vendido)
print("Total de unidades vendidas:", maior_quantidade)

Total de unidades por produto:
Produto
Produto 1     1117
Produto 10    1074
Produto 2     1170
Produto 3     1180
Produto 4     1006
Produto 5     1163
Produto 6     1163
Produto 7      993
Produto 8     1036
Produto 9      987
Name: Quantidade, dtype: int64

Produto mais vendido: Produto 3
Total de unidades vendidas: 1180


In [106]:
#Qual é a média da quantidade vendida para o Produto 8 no Estado de Pernambuco (PE)?
df_md_pe =  df[(df['Produto'] == 'Produto 8') & (df['Estado'] == 'PE')]
media_prod_pe = df_md_pe["Quantidade"].mean()
print(f"Média da quantidade vendida para o Produto 8 em Pernambuco: {media_prod_pe}")


Média da quantidade vendida para o Produto 8 em Pernambuco: 5.409090909090909


In [107]:
#Qual é a soma do valor da venda para todos os produtos vendidos para os estados de MG e SP?
filtro_mg_sp = df[df['Estado'].isin(['MG', 'SP'])]
soma_valor_total_mg_sp = filtro_mg_sp['Valor_Total'].sum()
print(f"soma do valor da venda para todos os produtos vendidos para os estados de MG e SP: {soma_valor_total_mg_sp}")

soma do valor da venda para todos os produtos vendidos para os estados de MG e SP: 43289.45


In [108]:
#Calcule a média da coluna "Valor Total" para todos os produtos que estão disponíveis nos estados de AL, BA, CE, MA, PB, PE, PI, RN, SE.
estados_nordeste = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']

# Filtrar o DataFrame para os estados do Nordeste
df_nordeste = df[df['Estado'].isin(estados_nordeste)]

# Calcular a média do Valor Total para esses estados
media_valor_total_nordeste = df_nordeste['Valor_Total'].mean()
print(f"Média do Valor Total nos estados do Nordeste: R$ {media_valor_total_nordeste:,.2f}")


Média do Valor Total nos estados do Nordeste: R$ 114.19
